In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
import sparse_dot_topn
import re

# Loading data

In [ ]:
df = pd.read_pickle('/mnt/data/mediascreen/sharedfolder/em/hrns_clean.gzip')

In [ ]:
df.head()

# Getting unique values

In [ ]:
%%time
id_unique_values = df.groupby(df['hrn_id']).agg({'hrn_type': lambda x: list(set(x)),
                                             'first_name': lambda x: list(set(x)),
                                             'middle_name': lambda x: list(set(x)),
                                             'last_name': lambda x: list(set(x)),
                                             'prefix': lambda x: list(set(x)),
                                             'suffix': lambda x: list(set(x)),
                                             'nationalities': lambda x: list(set(sum(x, []))),
                                             'countries_residence': lambda x: list(set(sum(x, []))),
                                             'dobs': lambda x: list(set(sum(x, []))),
                                             'genders': lambda x: list(set(sum(x, [])))
                                            })

In [ ]:
len(id_unique_values)
id_unique_values.head()

We will combine nationalities and countries into 1 column

In [ ]:
def get_gender(gender_list):
    if len(gender_list) == 0:
        gender = 'unknown'
    elif gender_list[0] in ('male', 'female'):
        gender = gender_list[0]
    else:
        gender = 'unknown'
    return gender

In [ ]:
id_unique_values['gender'] = id_unique_values['genders'].apply(lambda x: get_gender(x))

id_unique_values_details = id_unique_values[['nationalities', 'countries_residence', 'dobs', 'gender']]

id_unique_values_details['nationalities'] = (id_unique_values_details['nationalities'] + id_unique_values_details['countries_residence']).apply(lambda x: list(set(x)))

id_unique_values_details = id_unique_values_details[['nationalities', 'dobs', 'gender']]

In [ ]:
len(id_unique_values_details)
id_unique_values_details.head()

In [ ]:
id_unique_values_details.to_pickle('/mnt/data/mediascreen/sharedfolder/em/models/id_unique_values_details.pkl', compression= 'gzip')